In [1]:
import numpy as np
import pandas as pd
import pickle

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import nltk
import re
from nltk.tokenize import RegexpTokenizer
from sklearn.naive_bayes import MultinomialNB
from nltk.stem import WordNetLemmatizer
from sklearn import svm
from sklearn.svm import SVC  
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from nltk.corpus import stopwords
from sklearn.linear_model import SGDClassifier
import pickle

In [2]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/julioferreira/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/julioferreira/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/julioferreira/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [3]:
train = pd.read_csv('/home/cunha/covid19-sample/training.csv', sep=';')

In [4]:
test = pd.read_csv('/home/cunha/covid19-sample/test.csv', sep=';')

In [5]:
def rm_URLs(data):
    return re.sub('((www.[^s]+)|(https?://[^s]+))',' ',data)
train["text"] = train["text"].apply(lambda s: rm_URLs(s))

In [6]:
def rm_emoji(string):
    return string.encode('ascii', 'ignore').decode('ascii')
train["text"] = train["text"].apply(lambda s: rm_emoji(s))

In [7]:
train["text"]= train["text"].apply(lambda s: ' '.join(re.sub("[.,!?:;-='...@#_]", " ", s).split()))

In [8]:
def rm_repeated(string):
    return re.sub(r'(.)1+', r'1', string)
train['text'] = train['text'].apply(lambda x: rm_repeated(x))

In [9]:
train.loc[train.country_code!="US",'country_code'] = 0
train['country_code'] = train['country_code'].replace("US",1)

In [10]:
X = train.text
y = train.country_code

In [11]:
test.loc[test.country_code!="US",'country_code'] = 0
test['country_code'] = test['country_code'].replace("US",1)
H = test.text
Hy = test.country_code

In [12]:
sw = stopwords.words('english')
vectoriser = TfidfVectorizer(stop_words=sw)
vectoriser.fit(X)

TfidfVectorizer(stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours',
                            'ourselves', 'you', "you're", "you've", "you'll",
                            "you'd", 'your', 'yours', 'yourself', 'yourselves',
                            'he', 'him', 'his', 'himself', 'she', "she's",
                            'her', 'hers', 'herself', 'it', "it's", 'its',
                            'itself', ...])

In [13]:
mnb = MultinomialNB()
model = Pipeline([('vectorizer', vectoriser), ('mod', mnb)])
model.fit(X, y)
filename = 'final_model.sav'
pickle.dump(model, open(filename, 'wb'))

In [15]:
#predictions = vec_clf.predict(H)
#print(accuracy_score(Hy, predictions))